In [1]:
import warnings  # Hide all warnings here
warnings.filterwarnings("ignore")
from time import time
import mne
mne.utils.set_log_level('error')
import numpy as np
import local
from models.preprocessing import utils

In [2]:
d = "../../data/reinhartlab/multimodal/cg/Epochs/"
ext_in = "-epo.fif.gz"
box = utils.select_dataset(d,ext_in);box

In [3]:
freqs = [2,3,4,5,6,7,8,9,11,13,15,17,20,23,26,30,35,40,45,50,60,70,80,90,100]
n_cycles = 5

In [ ]:
from time import time
files = [d+f+ext_in for f in utils.get_selection(box)]
for f in files[1:]:
    s=time()
    enr = mne.read_epochs(f)
    picks=mne.pick_types(enr.info,eeg=True)
    # MAKE TFR BY CHANNEL
    tfr = mne.time_frequency.tfr_morlet(enr,[freqs[0]],n_cycles,picks=picks,return_itc=0,average=0,n_jobs=len(picks)).apply_baseline(mode='percent',baseline=(-.2,0))
    tfr.data = np.mean(tfr.data,axis=1,keepdims=True)
    for frq in freqs[1:]:
        tf = mne.time_frequency.tfr_morlet(enr,[frq],n_cycles,picks=picks,return_itc=0,average=0,n_jobs=len(picks)).apply_baseline(mode='percent',baseline=(-.2,0))
        tfr.data = np.dstack([tfr.data,np.mean(tf.data,axis=1,keepdims=True)])
    tfr.freqs = freqs
    tfr.info['ch_names'] = ['Cz']
    tfr.info['chs'] = [c for c in tfr.info['chs'] if c.get('ch_name','')=='Cz']
    tfr.info['nchan'] = 1
    tfr.event_id = {k.replace('/','_'):v for k,v in tfr.event_id.items()}
    tfr.save(f[:-len(ext_in)]+'.aggregate.2to100-tfr.h5', overwrite=True)
    print(f[len(d):-len(ext_in)]+", duration: {:.2f}s".format(time()-s))

MulitModal_EEG_058CG.response, duration: 999.38s
MulitModal_EEG_058CG.stimulus, duration: 1015.92s
MulitModal_EEG_069CG.feedback, duration: 708.71s
MulitModal_EEG_069CG.response, duration: 592.72s
MulitModal_EEG_069CG.stimulus, duration: 698.41s
MulitModal_EEG_076CG.feedback, duration: 684.96s
MulitModal_EEG_076CG.response, duration: 335.03s
MulitModal_EEG_076CG.stimulus, duration: 679.09s
MulitModal_EEG_078CG.feedback, duration: 1010.92s
MulitModal_EEG_078CG.response, duration: 999.35s
MulitModal_EEG_078CG.stimulus, duration: 1007.73s
MulitModal_EEG_094CG.feedback, duration: 705.66s
MulitModal_EEG_094CG.response, duration: 698.22s
MulitModal_EEG_094CG.stimulus, duration: 710.85s
MulitModal_EEG_095part2CG.feedback, duration: 302.18s
MulitModal_EEG_095part2CG.response, duration: 301.32s
MulitModal_EEG_095part2CG.stimulus, duration: 302.68s
MulitModal_EEG_099CG.feedback, duration: 981.67s
MulitModal_EEG_099CG.response, duration: 983.11s
MulitModal_EEG_099CG.stimulus, duration: 991.88s
Mu

In [ ]:
def binned_mean_func(ts):
    time_slice = lambda t,s: np.all([t>s[0],t<=s[1]],axis=0)
    get_slice_mean = lambda x,t: np.mean(x.data[:,:,:,time_slice(x.times,t)],axis=3,keepdims=True)
    return lambda tfr: np.concatenate([get_slice_mean(tfr,t) for t in zip(ts[:-1],ts[1:])],axis=3)

from time import time
files = [d+f+ext_in for f in utils.get_selection(box)]
ts=np.array(range(-10,21))/10
binned_mean = binned_mean_func(ts)
for f in files[16:]:
    s=time()
    enr = mne.read_epochs(f)
    picks=mne.pick_types(enr.info,eeg=True)
    # MAKE TFR BY CHANNEL
    tfr = mne.time_frequency.tfr_morlet(enr,[freqs[0]],n_cycles,picks=picks,return_itc=0,average=0,n_jobs=len(picks)).apply_baseline(mode='percent',baseline=(-.2,0))
    tfr.data = np.concatenate([get_slice_mean(tfr,t) for t in zip(ts[:-1],ts[1:])],axis=3)
    for frq in freqs[1:]:
        tf = mne.time_frequency.tfr_morlet(enr,[frq],n_cycles,picks=picks,return_itc=0,average=0,n_jobs=len(picks)).apply_baseline(mode='percent',baseline=(-.2,0))
        tfr.data = np.dstack([tfr.data,np.concatenate([get_slice_mean(tf,t) for t in zip(ts[:-1],ts[1:])],axis=3)])
    tfr.freqs = freqs
    tfr.times = ts[:-1]
    tfr.event_id = {k.replace('/','_'):v for k,v in tfr.event_id.items()}
    tfr.save(f[:-len(ext_in)]+'.binned.2to100-tfr.h5', overwrite=True)
    print(f[len(d):-len(ext_in)]+", duration: {:.2f}s".format(time()-s))

MulitModal_EEG_095part2CG.response, duration: 302.38s
MulitModal_EEG_095part2CG.stimulus, duration: 296.60s
MulitModal_EEG_099CG.feedback, duration: 990.80s
MulitModal_EEG_099CG.response, duration: 990.41s
MulitModal_EEG_099CG.stimulus, duration: 976.61s
MulitModal_EEG_102CG.feedback, duration: 1009.30s
MulitModal_EEG_102CG.response, duration: 996.39s
MulitModal_EEG_102CG.stimulus, duration: 1008.65s
MulitModal_EEG_113CG_2.feedback, duration: 120.94s
MulitModal_EEG_113CG_2.response, duration: 119.05s
MulitModal_EEG_113CG_2.stimulus, duration: 119.42s
MultiModal_EEGTest_054CG.feedback, duration: 1000.97s
MultiModal_EEGTest_054CG.response, duration: 999.88s
MultiModal_EEGTest_054CG.stimulus, duration: 1004.48s
MultiModal_EEGTest_066CG.feedback, duration: 1006.71s
MultiModal_EEGTest_066CG.response, duration: 992.35s
MultiModal_EEGTest_066CG.stimulus, duration: 1015.58s
MultiModal_EEGTest_071CG.feedback, duration: 773.85s
MultiModal_EEGTest_071CG.response, duration: 729.00s
MultiModal_EEGT